In [2]:
import pandas as pd

In [3]:
import julia

julia.install()

In [4]:
from julia import Main

In [5]:
Main.eval(
    """
using DataFrames
using Pandas

using SEA
"""
)

JuliaError: Exception 'ArgumentError: Package DataFrames not found in current path:
- Run `import Pkg; Pkg.add("DataFrames")` to install the DataFrames package.
' occurred while calling julia code:

using DataFrames
using Pandas

using SEA


In [ ]:
element_score_ = pd.read_csv(
    "~/code/julia/SEA.jl/notebook/data/gene_score.tsv",
    sep="\t",
    index_col=0,
    squeeze=True,
)

element_x_sample = pd.concat(
    (element_score_, element_score_ * 10, element_score_ / 10), axis=1
).reset_index()

element_x_sample.columns = (
    "Sample {}".format(i) for i in range(element_x_sample.shape[1])
)

set_to_element_ = Main.eval(
    """
SEA.read_gmt("/Users/kwat/code/julia/SEA.jl/notebook/data/c2.all.v7.1.symbols.gmt")
"""
)

set_element_ = set_to_element_["COLLER_MYC_TARGETS_UP"]

In [ ]:
def send(element_score_, set_element, set_to_element_, element_x_sample):

    if element_score_ is not None:

        Main.element_ = list(element_score_.index)

        Main.score_ = list(element_score_)

    if set_element_ is not None:

        Main.set_element_ = list(set_element_)

    if set_to_element_ is not None:

        Main.set_to_element_ = set_to_element_

        Main.eval(
            """
        set_to_element_ = convert(Dict{String, Vector{String}}, set_to_element_)
        """
        )

    if element_x_sample is not None:

        Main.element_x_sample = element_x_sample

        Main.eval(
            """
        element_x_sample = DataFrames.DataFrame(Pandas.DataFrame(element_x_sample))
        """
        )

In [ ]:
def score_1_sample_1_set(element_score_, set_element_, sort=True, plot=True):

    send(element_score_, set_element_, None, None)

    Main.sort = sort

    Main.plot = plot

    result = Main.eval(
        """
    score_set(
        element_,
        score_,
        set_element_;
        sort = sort,
        plot = plot,
    )
    """
    )

    return result


score_1_sample_1_set(element_score_, set_element_, sort=True, plot=False)

In [ ]:
def score_1_sample_n_set(element_score_, set_to_elements, sort=True):

    send(element_score_, None, set_to_element_, None)

    Main.sort = sort

    set_to_retsult = Main.eval(
        """
    score_set(
        element_,
        score_,
        set_to_element_;
        sort = sort,
    )
    """
    )

    return set_to_retsult


score_1_sample_n_set(element_score_, set_element_, sort=True)

In [ ]:
def score_n_sample_n_set(element_x_sample, set_to_element_, n_job=1):

    send(None, None, set_to_element_, element_x_sample)

    Main.n_job = n_job

    set_x_sample = Main.eval(
        """
    Pandas.DataFrame(score_set(
        element_x_sample,
        set_to_element_;
        n_job = n_job,
    ))
    """
    )

    return set_x_sample.set_index("Set")


score_n_sample_n_set(element_x_sample, set_to_element_, n_job=1)